Input File Creation
===========
First let's start with some tools to create input files for a given deployment schedule.

In [77]:
import os
import uuid
import json
import subprocess
from math import ceil
from copy import deepcopy

import numpy as np
import pandas as pd
import cymetric as cym
%matplotlib inline
import matplotlib.pyplot as plt

In [42]:
with open('once-through.json') as f:
    BASE_SIM = json.load(f)
DURATION = BASE_SIM['simulation']['control']['duration']
YEARS = ceil(DURATION / 12)
MONTH_SHUFFLE = (0, 6, 9, 3, 7, 5, 11, 1, 4, 8, 10, 2)
NULL_SCHEDULE = {'build_times': [{'val': 1}], 
                 'n_build': [{'val': 0}], 
                 'prototypes': [{'val': 'LWR'}]}
LWR_PROTOTYPE = {'val': 'LWR'}
OPT_H5 = 'opt.h5'

In [20]:
BASE_SIM['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

In [26]:
def deploy_inst_schedule(Theta):
    if np.sum(Theta) == 0: 
        return NULL_SCHEDULE
    sched = {}
    build_times = sched['build_times'] = []
    n_build = sched['n_build'] = []
    prototypes = sched['prototypes'] = []
    m = 0
    for i, theta in enumerate(Theta):
        if theta <= 0:
            continue
        build_times.append(i*12 + MONTH_SHUFFLE[m])
        n_build.append(theta)
        prototypes.append(LWR_PROTOTYPE)
        m = (m + 1) % 12
    return sched

def make_sim(Theta, fname='sim.json'):
    sim = deepcopy(BASE_SIM)
    inst = sim['simulation']['region']['institution']
    inst['config']['DeployInst'] = deploy_inst_schedule(Theta)
    with open(fname, 'w') as f:
        json.dump(sim, f)
    return sim

In [37]:
s = make_sim([])

In [38]:
s['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

Simulate
=========
Now let's build some tools to run simulations and extract a GWe time series.

In [97]:
def run(fname='sim.json', out=OPT_H5):
    """Runs a simulation and returns the sim id."""
    cmd = ['cyclus', '-o', out, fname]
    proc = subprocess.run(cmd, check=True, universal_newlines=True, stdout=subprocess.PIPE)
    simid = proc.stdout.rsplit(None, 1)[-1]
    return simid

ZERO_GWE = pd.DataFrame({'GWe': np.zeros(YEARS)}, index=np.arange(YEARS))
ZERO_GWE.index.name = 'Time'

def extract_gwe(simid, out=OPT_H5):
    """Computes the annual GWe for a simulation."""
    db = cym.dbopen(out)
    evaler = cym.Evaluator(db)
    raw = evaler.eval('TimeSeriesPower', conds=[('SimId', '==', uuid.UUID(simid))])
    ano = pd.DataFrame({'Time': raw.Time.apply(lambda x: x//12), 
                        'GWe': raw.Value.apply(lambda x: 1e-3*x/12)})
    gwe = ano.groupby('Time').sum()
    gwe = (gwe + ZERO_GWE).fillna(0.0)
    return np.array(gwe.GWe)

Initialize Optimization
===============
Now let's start with a couple of simple simulations

In [54]:
if os.path.isfile(OPT_H5):
    os.remove(OPT_H5)

In [36]:
N = np.asarray(np.ceil(4*(1.01)**np.arange(YEARS)), dtype=int)
N

array([4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7,
       7, 7, 7, 7])

In [55]:
Theta0 = [0] * YEARS
make_sim(Theta0)
simid = run()

In [98]:
gwe = extract_gwe(simid)

In [99]:
gwe

array([ 85.16666667,  91.08333333,  88.33333333,  86.33333333,
        84.        ,  81.25      ,  79.25      ,  76.91666667,
        74.16666667,  72.33333333,  69.41666667,  67.5       ,
        65.08333333,  62.33333333,  60.58333333,  57.91666667,
        55.5       ,  53.33333333,  50.91666667,  48.41666667,
        46.25      ,  43.58333333,  41.58333333,  39.08333333,
        36.58333333,  34.25      ,  32.        ,  29.75      ,
        27.58333333,  25.16666667,  22.75      ,  20.5       ,
        18.        ,  15.75      ,  13.41666667,  10.91666667,
         8.66666667,   6.25      ,   3.91666667,   1.66666667,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ])

In [73]:
import uuid

In [76]:
uuid.UUID('{' + simid + '}')

UUID('4fc7fd42-c7e6-4a14-b13b-068d6c576eb6')